In [1260]:
import pandas as pd
import sys, os
import tensorflow as tf
import shape_helper
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [1274]:

#the following bits are basically just to deal with translations and rotations in a pretty efficient manner. Note the 
#translation+removal functions have no for loops so it's pretty quick 
#Let's make the goal shape into a bunch of 1's and 0's representing where we want pieces
def mattoones(goalshape1):
    for i in range(len(goalshape1)):
        for k in range(len(np.transpose(goalshape1))):
            if goalshape1[i][k]!='0.0' and goalshape1[i][k]!=0:
                goalshape1[i][k]=1
    return goalshape1
#Remove all rows/columns at edges with only 0's in order to standardize calculations 
def translator(matrix):
    sums=0;
    while(sum(matrix[0])==0):
        if (sum(matrix[0]==0)):
            matrix=np.delete(matrix,0,0)

    i=len(matrix)-1
    while(sums==0):
        sums=sum(matrix[i]);
        if(sums==0):
            matrix=np.delete(matrix,i,0)
        i=i-1
    return(matrix)

#(*use the above function but for columns too*)
def removal(matrixer):
    c=translator(matrixer);
    c=translator(np.transpose(c))
    return(np.transpose(c))
def sqdiff(m1,m2):
    return (sum(sum((m1-m2)*(m1-m2))))
#I think this lets us visualize chains as 1/0s instead of HP but it's not very useful 
def graphicchainno(chain):
    boundary=np.floor(np.sqrt(len(chain))*2);
    lattice=np.ndarray.tolist(np.zeros([int(boundary),int(boundary)]))
    for i in range(len(chain)):
        lattice[chain[i][0]][chain[i][1]]=1
    lattice=np.array(lattice)
    return(lattice)

In [1275]:
x=1
model=keras.Sequential([
    #Entry block: create four masks, and run
    layers.Conv2D(filters=128,kernel_size=3,activation="relu",padding='same',input_shape=[13,13,1]),
    layers.MaxPool2D(),
    layers.Dropout(.3),
    #Block 2
    layers.Conv2D(filters=128, kernel_size=3,activation="relu",padding='same'),
    layers.MaxPool2D(),layers.Dropout(.3),

    #Third hidden layer
    layers.Conv2D(filters=128,kernel_size=3,activation="relu",padding='same'),
    #output 
    layers.Flatten(),
    layers.Dense(units=169,activation="relu"),
    layers.Dense(units=169,activation="sigmoid")
])
model.summary()

Model: "sequential_53"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_162 (Conv2D)         (None, 13, 13, 128)       1280      
                                                                 
 max_pooling2d_108 (MaxPooli  (None, 6, 6, 128)        0         
 ng2D)                                                           
                                                                 
 dropout_69 (Dropout)        (None, 6, 6, 128)         0         
                                                                 
 conv2d_163 (Conv2D)         (None, 6, 6, 128)         147584    
                                                                 
 max_pooling2d_109 (MaxPooli  (None, 3, 3, 128)        0         
 ng2D)                                                           
                                                                 
 dropout_70 (Dropout)        (None, 3, 3, 128)       

1+1

In [1276]:
#set datasets 
#from sklearn.model_selection import train_test_split
import random
import shape_helper
df=pd.read_csv('/Users/joshuatsai/Documents/GitHub/SigmaFold/library/sequence_training.csv')
targets=df['path']
startshape=df['shape_mapping']
x_train=[]
y_train=[]
validtargets=[]
validshapes=[]
arraysize=13
#create our array entries 
for i in range(len(startshape)):
    c=shape_helper.deserialize_path(df['path'][i])
    d=path_to_shape(c,arraysize)
    position=start_to_class(d[2],arraysize)
    targets[i]=position
    startshape[i]=mattoones(d[0])

#onehotencode position
onehottable=np.zeros([len(targets),arraysize*arraysize])
for i in range(len(targets)):
    onehottable[i][targets[i]]=1
    y_train.append(targets[i])

#onehottablepos=np.zeros([len(targets),arraysize*arraysize])
for i in range(len(targets)):
    x_train.append(startshape[i])
   # for x in range(len(startshape[i])):
       # for y in range(len(startshape[i][1])):
           # if startshape[i][x][y]!=0:
                #onehottablepos[i][x*len(startshape[i]+y)]=1



for i in range(len(startshape)):
    prob=random.random()
    eve=len(startshape)
    if prob<.1:

        validshapes.append(startshape[eve-i])
        validtargets.append(targets[eve-i])
        np.delete(x_train,eve-i,0)
        np.delete(onehottable,eve-i,0)
        
#startshapes=tf.stack(startshape,np.float32)
onehottableval=np.zeros([len(validtargets),arraysize*arraysize])
for i in range(len(validtargets)):
    onehottableval[i][validtargets[i]]=1
    
trainx=tf.convert_to_tensor(x_train)
trainy=tf.convert_to_tensor(onehottable)
valx=tf.convert_to_tensor(validshapes)
valy=tf.convert_to_tensor(onehottableval)


#trainy=[tf.from_numpy(item).float() for item in startshape]



startpos ==> (1, -1)
startpos ==> (1, 2)
startpos ==> (1, -1)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-1, 1)
startpos ==> (-1, 0)
startpos ==> (-1, 0)
startpos ==> (-1, 0)
startpos ==> (0, 3)
startpos ==> (0, -2)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-1, 0)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-1, 1)
startpos ==> (-1, 0)
startpos ==> (-1, 0)
startpos ==> (-1, 0)
startpos ==> (0, 3)
startpos ==> (0, -2)
startpos ==> (0, 3)
startpos ==> (0, -2)
startpos ==> (0, 3)
startpos ==> (0, -2)
startpos ==> (-1, 1)
startpos ==> (-1, 0)
startpos ==> (0, 3)
startpos ==> (0, -2)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (1, 1)
startpos ==> (1, 0)
startpos ==> (1, 1)
startpos ==> (1, 0)
startpos ==> (1, 1)
startpos ==> (1, 0)
startpos ==> (1, 1)
startpos ==> (1, 0)
startp

KeyError: 6254

In [1264]:
np.mean(targets)

array([85.82587144])

In [1281]:
#set datasets 
#from sklearn.model_selection import train_test_split
import random
import shape_helper
df=pd.read_csv('/Users/joshuatsai/Documents/GitHub/SigmaFold/library/sequence_training.csv')

targets=[]
startshape=[]
lengths=df['length']
tots=len(targets)

        
x_train=[]
y_train=[]
validtargets=[]
validshapes=[]
onesizecheck=[]
arraysize=13

#create our array entries 
for i in range(len(lengths)):
    # if you want to exclude certain lengths can do 
    #if lengths[i]!=excluded length: 
    
        
    c=shape_helper.deserialize_path(df['path'][i])
    d=path_to_shape(c,arraysize)
    position1=start_to_class(d[2],arraysize)
    position2=start_to_class(d[3],arraysize)
    targets.append(position1)
    targets.append(position2)
    startshape.append(mattoones(d[1]))
    startshape.append(mattoones(d[1]))
#else:
#onesizecheck.append(mattoones(d[1]))
#onehotencode position
#print(len(startshape))
onehottable=np.zeros([len(targets),arraysize*arraysize])
for i in range(len(targets)):
    onehottable[i][targets[i]]=1
    y_train.append(targets[i])

#onehottablepos=np.zeros([len(targets),arraysize*arraysize])
for i in range(len(targets)):
    x_train.append(startshape[i])
   # for x in range(len(startshape[i])):
       # for y in range(len(startshape[i][1])):
           # if startshape[i][x][y]!=0:
                #onehottablepos[i][x*len(startshape[i]+y)]=1



for i in range(len(startshape)):
    prob=random.random()
    eve=len(startshape)
    if prob<.1:

        validshapes.append(startshape[eve-1-i])
        validtargets.append(targets[eve-1-i])
        x_train=np.delete(x_train,eve-1-i,0)
        onehottable=np.delete(onehottable,eve-1-i,0)

#startshapes=tf.stack(startshape,np.float32)
onehottableval=np.zeros([len(validtargets),arraysize*arraysize])
for i in range(len(validtargets)):
    onehottableval[i][validtargets[i]]=1
    
trainx=tf.convert_to_tensor(x_train)
trainy=tf.convert_to_tensor(onehottable)
valx=tf.convert_to_tensor(validshapes)
valy=tf.convert_to_tensor(onehottableval)


#trainy=[tf.from_numpy(item).float() for item in startshape]



startpos ==> (1, -1)
startpos ==> (1, 2)
startpos ==> (1, -1)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-1, 1)
startpos ==> (-1, 0)
startpos ==> (-1, 0)
startpos ==> (-1, 0)
startpos ==> (0, 3)
startpos ==> (0, -2)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-1, 0)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-1, 1)
startpos ==> (-1, 0)
startpos ==> (-1, 0)
startpos ==> (-1, 0)
startpos ==> (0, 3)
startpos ==> (0, -2)
startpos ==> (0, 3)
startpos ==> (0, -2)
startpos ==> (0, 3)
startpos ==> (0, -2)
startpos ==> (-1, 1)
startpos ==> (-1, 0)
startpos ==> (0, 3)
startpos ==> (0, -2)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (1, 1)
startpos ==> (1, 0)
startpos ==> (1, 1)
startpos ==> (1, 0)
startpos ==> (1, 1)
startpos ==> (1, 0)
startpos ==> (1, 1)
startpos ==> (1, 0)
startp

In [1266]:
len(valy)

1241

In [1280]:
c

0       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...
1       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...
2       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...
3       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...
4       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...
                              ...                        
6249    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...
6250    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...
6251    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...
6252    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...
6253    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...
Name: shape_mapping, Length: 6254, dtype: object

In [1282]:
#code to select a validation dataset and allow for multiple answers 
c=df['shape_mapping']
d=df['path']
list = [v for o,v in c.items()]
starts=[v for o,v in d.items()]
actualshapes=set(c)
valshapes=[]

#here we define our validation data set. You can change this to be all n of length x 
for shape in actualshapes:
    prob=random.random()
    if prob<.04:
        valshapes.append(shape)
#here we remove the validation from the training     
for shape in valshapes:
    actualshapes.remove(shape)
#here we remove validation from training 
def Convert(lst):
    res_dct = {lst[i]:[] for i in range(0, len(lst))}
    return res_dct
valstarts=Convert(valshapes)
for i in range(len(c)):
    for j in range(len(valshapes)):
        if valshapes[j]==list[len(c)-1-i]:
            list=np.delete(list,len(c)-1-i)
            starts=np.delete(starts,len(c)-1-i)
            valstarts[valshapes[j]].append(starts[len(c)-1-i])

# in summary we now have 
#valshapes=list of validation shapes. #valstarts=list of paths
#list=list of training stuff #starts=list of training paths


        


In [1283]:
#now we convert our training data into the correct format
targets=[]
startshape=[]
for i in range(len(list)):
    # if you want to exclude certain lengths can do 
    #if lengths[i]!=excluded length: 
    
        
    c=shape_helper.deserialize_path(starts[i])
    d=path_to_shape(c,arraysize)
    position1=start_to_class(d[2],arraysize)
    position2=start_to_class(d[3],arraysize)
    targets.append(position1)
    targets.append(position2)
    startshape.append(mattoones(d[1]))
    startshape.append(mattoones(d[1]))
onehottable=np.zeros([len(targets),arraysize*arraysize])
for i in range(len(targets)):
    onehottable[i][targets[i]]=1
x_train=[]
for i in range(len(targets)):
    x_train.append(startshape[i])
x_train=tf.convert_to_tensor(x_train)
onehottable=tf.convert_to_tensor(onehottable)
#input for training is x_train. Output for training is onehottable


startpos ==> (1, -1)
startpos ==> (1, 2)
startpos ==> (1, -1)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-1, 1)
startpos ==> (-1, 0)
startpos ==> (-1, 0)
startpos ==> (-1, 0)
startpos ==> (0, 3)
startpos ==> (0, -2)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-1, 0)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-1, 1)
startpos ==> (-1, 0)
startpos ==> (-1, 0)
startpos ==> (-1, 0)
startpos ==> (0, 3)
startpos ==> (0, -2)
startpos ==> (0, 3)
startpos ==> (0, -2)
startpos ==> (0, 3)
startpos ==> (0, -2)
startpos ==> (-1, 1)
startpos ==> (-1, 0)
startpos ==> (0, 3)
startpos ==> (0, -2)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (-2, 0)
startpos ==> (-2, 1)
startpos ==> (1, 1)
startpos ==> (1, 0)
startpos ==> (1, 1)
startpos ==> (1, 0)
startpos ==> (1, 1)
startpos ==> (1, 0)
startpos ==> (1, 1)
startpos ==> (1, 0)
startp

In [1284]:
#now we take valstarts which is a list of a list of paths and convert each path into a start
valys=Convert(valshapes)
count=0
for shape in valstarts:
    
    for x in valstarts[shape]:
        c=shape_helper.deserialize_path(x)
        d=path_to_shape(c,arraysize)
        
        position1=start_to_class(d[2],arraysize)
        
        
        position2=start_to_class(d[3],arraysize)
        valys[shape].append(position1[0])
        valys[shape].append(position2[0])
        
valx=[]
count=0
for shape in valstarts:
    if len(valstarts[shape])>0:

        c=shape_helper.deserialize_path(valstarts[shape][0])
        d=path_to_shape(c,arraysize)
    
        valx.append(d[1])
valhotencode=np.zeros([len(valys),arraysize*arraysize])
for i in range(len(valshapes)):
    for starting in valys[valshapes[i]]:
        valhotencode[i][starting]=1
valx=tf.convert_to_tensor(valx)
valhotencode=valhotencode
#input for validation is valx output is valhotencode
        

startpos ==> (1, 0)
startpos ==> (1, 0)
startpos ==> (1, 0)
startpos ==> (1, 0)
startpos ==> (1, -1)
startpos ==> (1, 0)
startpos ==> (0, 1)
startpos ==> (1, -1)
startpos ==> (2, 0)
startpos ==> (0, -1)
startpos ==> (0, -1)
startpos ==> (0, -1)
startpos ==> (0, 1)
startpos ==> (0, -1)
startpos ==> (0, -1)
startpos ==> (0, -1)
startpos ==> (0, 1)
startpos ==> (0, 1)
startpos ==> (0, 1)
startpos ==> (0, 1)
startpos ==> (0, 0)
startpos ==> (0, 0)
startpos ==> (0, 0)
startpos ==> (0, 0)
startpos ==> (-1, 1)
startpos ==> (-1, 1)
startpos ==> (-1, 1)
startpos ==> (-1, 1)
startpos ==> (-1, 1)
startpos ==> (-1, 1)
startpos ==> (-1, 1)
startpos ==> (-2, 0)
startpos ==> (-2, 0)
startpos ==> (-1, 1)
startpos ==> (-2, 0)
startpos ==> (-2, 0)
startpos ==> (-1, 1)
startpos ==> (-1, 1)
startpos ==> (-1, 1)
startpos ==> (0, 2)
startpos ==> (0, 2)
startpos ==> (0, 2)
startpos ==> (0, 2)
startpos ==> (0, 2)
startpos ==> (0, 2)
startpos ==> (1, 2)
startpos ==> (0, 1)
startpos ==> (0, 1)
startpos ==> (0, 

In [1277]:
import math as math
def path_to_shape(path,grid_sizes):
    grid = np.asarray([[0]*grid_sizes]*grid_sizes, dtype=int) # actual grid for mapping
    temp_grid = np.asarray([[0]*grid_sizes]*grid_sizes, dtype=int) # temp grid to hold the array before alignment
    for coord in path:
        temp_grid[coord[1]+round(grid_sizes/2)][coord[0]+ round(grid_sizes/2)] = 1
        if coord==path[len(path)-1]:
            temp_grid[coord[1]+round(grid_sizes/2)][coord[0]+ round(grid_sizes/2)] = 3
            # path transferred onto grid but uncentered
    temp_grid[round(grid_sizes/2)][round(grid_sizes/2)] = 2  

    temp_grid = np.pad(temp_grid, 1) # zero padding to avoid multiplying by 0 when calculating moments

    # find centroid of temp_grid
    m_00 = len(path) # non-zero residues
    m_01 = 0
    for row_n, row in enumerate(temp_grid):
        if np.any(row!=0):
            m_01 += row_n * np.count_nonzero(row)
    m_10 = 0
    for col_n in range(np.shape(temp_grid)[1]):
        if np.any(temp_grid[:, col_n]!=0):
            m_10 += col_n *  np.count_nonzero(temp_grid[:, col_n])

    # coordinates of centroid
    n_centroid = math.floor((m_10/m_00))
    m_centroid = math.floor((m_01/m_00))
    centroid = (m_centroid, n_centroid)
    #print("centroid ==>", centroid)
    # align temp_grid onto grid
    dev_m = round(grid_sizes/2)-centroid[0]
    dev_n = round(grid_sizes/2)-centroid[1]
    coord_list = np.nonzero(temp_grid)
    print("startpos ==>", (dev_m+1,dev_n+1))
    x,y=np.where(temp_grid==2)
    start= (x+dev_m,y+dev_n)
    x,y=np.where(temp_grid==3)
    end=(x+dev_m,y+dev_n)
    
    for i in range(len(coord_list[0])):
        grid[coord_list[0][i]+dev_m][coord_list[1][i]+dev_n] = 1
        #if i==0:
            #grid[coord_list[0][i]+dev_m][coord_list[1][i]+dev_n] = 2
    
    return temp_grid, grid,start,end
def start_to_class(coords,grid_size):
    return coords[0]*grid_size+coords[1]

In [1271]:
trainy

<tf.Tensor: shape=(11267, 169), dtype=float64, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])>

In [1272]:
from tensorflow import keras
from tensorflow.keras import layers
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)
model.compile(
    optimizer=tf.keras.optimizers.Adam(epsilon=.01),
    loss='CategoricalCrossentropy',metrics=['AUC']
)

In [1273]:
history = model.fit(
    x_train,onehottable,epochs=50
)


Epoch 1/50
375/375 [==============================] - 3s 8ms/step - loss: 3.5070 - auc: 0.9201
Epoch 2/50
375/375 [==============================] - 3s 9ms/step - loss: 2.9346 - auc: 0.9563
Epoch 3/50
214/375 [================>.............] - ETA: 1s - loss: 2.8659 - auc: 0.9576

KeyboardInterrupt: 

In [1237]:
xk=model.predict(valx)

1/1 [==============================] - 0s 14ms/step


In [1238]:
thresx=threshold(xk)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [1248]:
acc(thresx,valhotencode)/len(thresx)

0.8421052631578947

In [1250]:
for i in range(len(thresx)):
    print(np.where(thresx[i]==1))

(array([84]),)
(array([59]),)
(array([85]),)
(array([59]),)
(array([59]),)
(array([72]),)
(array([110]),)
(array([97]),)
(array([97]),)
(array([85]),)
(array([71]),)
(array([97]),)
(array([85]),)
(array([83]),)
(array([85]),)
(array([85]),)
(array([110]),)
(array([97]),)
(array([97]),)


In [ ]:
thresx=threshold(x)
correct=acc(thresx,valy)

In [1241]:
def acc(array1,array2):
    total=0
    for i in range(len(array1)):
        if len(np.where(array1[i]==1)[0])==1:
            if array2[i][np.where(array1[i]==1)]==1:
            
                total=1+total
    return total
def threshold(array1):
    lop=np.zeros([len(array1),len(array1[1])])
    for i in range(len(array1)):
        lop[i][np.where(array1[i]==max(array1[i]))]=1
    print(lop)
    return lop



In [ ]:
trues=np.zeros([13,13])
for i in range(len(thresx)):
    if len(np.where(thresx[i]==1)[0])==1:
        trues[math.floor(np.where(valy[i]==1)[0][0]/13)][np.where(valy[i]==1)[0][0]%13]=trues[math.floor(np.where(valy[i]==1)[0][0]/13)][np.where(valy[i]==1)[0][0]%13]+1
    #if i<len(thresx)-1:
        #if(np.where(trainy[i]==1)==np.where(thresx[i+1]==1)):
           # print(i)
            #cor=cor+1
    #if i>0:
        #if(np.where(trainy[i]==1)==np.where(thresx[i-1]==1)):
            #print(i)
            #cor=cor+1
zs=np.zeros([13,13])
for i in range(len(thresx)):
    if len(np.where(thresx[i]==1)[0])==1:
        zs[math.floor(np.where(thresx[i]==1)[0][0]/13)][np.where(thresx[i]==1)[0][0]%13]=zs[math.floor(np.where(thresx[i]==1)[0][0]/13)][np.where(thresx[i]==1)[0][0]%13]+1
    #if i<len(thresx)-1:
        #if(np.where(trainy[i]==1)==np.where(thresx[i+1]==1)):
           # print(i)
            #cor=cor+1
    #if i>0:
        #if(np.where(trainy[i]==1)==np.where(thresx[i-1]==1)):
            #print(i)
            #cor=cor+1


In [ ]:
#Our final accuracy statement
correct/len(valy)


In [ ]:
print(trues)
print(zs)

In [ ]:
cor=0
for i in range(len(thresx)):
    if(np.where(valy[i]==1)==np.where(thresx[i]==1)):
        print(i)
        cor=cor+1
    #if i<len(thresx)-1:
        #if(np.where(trainy[i]==1)==np.where(thresx[i+1]==1)):
           # print(i)
            #cor=cor+1
    #if i>0:
        #if(np.where(trainy[i]==1)==np.where(thresx[i-1]==1)):
            #print(i)
            #cor=cor+1
cor

In [ ]:
for i in range(len(thresx)):
    
    print(np.where(thresx[i]==1))

In [ ]:
np.where(thresx[5]==1)[0]